In [58]:
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 200)

import sys
import os
import lightgbm as lgbm
import catboost as ctb

from typing import Optional
from urllib.request import urlretrieve

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from ml_investment.utils import load_config, bound_filter_foo_gen
from ml_investment.features import QuarterlyFeatures, BaseCompanyFeatures, \
                                   FeatureMerger, DailyAggQuarterFeatures, \
                                   QuarterlyDiffFeatures, RelativeGroupFeatures
from ml_investment.targets import DailyDiffTarget
from ml_investment.models import GroupedOOFModel, EnsembleModel, LogExpModel
from ml_investment.metrics import median_absolute_relative_error, median_abs_diff
from ml_investment.pipelines import Pipeline
from ml_investment.download_scripts import download_sf1, download_commodities

In [ ]:
config = load_config()

In [3]:
config

{'sf1_data_path': '/Users/artem/.ml_investment/data/sf1',
 'yahoo_data_path': '/Users/artem/.ml_investment/data/yahoo',
 'commodities_data_path': '/Users/artem/.ml_investment/data/commodities',
 'daily_bars_data_path': '/Users/artem/.ml_investment/data/daily_bars',
 'models_path': '/Users/artem/.ml_investment/models',
 'out_path': '/Users/artem/.ml_investment/data/out'}

In [91]:
tickers_df = pd.read_csv(f'{config["sf1_data_path"]}/tickers.zip')
tickers_df = tickers_df[
    (tickers_df['currency'] == 'USD')
    & (tickers_df['scalemarketcap'].apply(lambda x: x in ["4 - Mid", "5 - Large", "6 - Mega"]))
#     & (tickers_df['scalerevenue'].apply(lambda x: x in ['6 - Mega']))
]
tickers = tickers_df['ticker'].values
tickers_df['scalerevenue'].value_counts()

3 - Small    3399
4 - Mid      2851
5 - Large    1377
2 - Micro     878
1 - Nano      205
6 - Mega       40
Name: scalerevenue, dtype: int64

In [6]:
OUT_NAME = 'quantile_diff_sf1_v2'
DATA_SOURCE='sf1'
CURRENCY = 'USD'
MAX_BACK_QUARTER = 20
MIN_BACK_QUARTER = 0
MAX_TARGET_BOUND = 1.5
MIN_TARGET_BOUND = -0.9
BAGGING_FRACTION = 0.7
MODEL_CNT = 20
FOLD_CNT = 5
QUARTER_COUNTS = [2, 4, 10]
COMPARE_QUARTER_IDXS = [1, 4]
AGG_DAY_COUNTS = [100, 200, 400, 800]
SCALE_MARKETCAP = ["4 - Mid", "5 - Large", "6 - Mega"]
CAT_COLUMNS = ["sector", "sicindustry"]
QUARTER_COLUMNS = [
    "revenue",
    "netinc",
    "ncf",
    "assets",
    "ebitda",
    "debt",
    "fcf",
    "gp",
    "workingcapital",
    "cashneq",
    "rnd",
    "sgna",
    "ncfx",
    "divyield",
    "currentratio",
    "netinccmn",
]
DEV_COLUMNS = [
    'rnd_invcap',
    'capex_invcap',
    'ebit_invcap',
    'ev_ebitda',
    'ev_ebit',
    'debt_equity',
    'grossmargin_ebitdamargin',
    'debt_ebit',
]
COMMODITIES_CODES = [
    'LBMA/GOLD',
    'JOHNMATT/PALL',
]

In [7]:
def _check_download_data():
    if not os.path.exists(config['sf1_data_path']):
        print('Downloading sf1 data')
        download_sf1.main()
        
    if not os.path.exists(config['commodities_data_path']):
        print('Downloading commodities data')
        download_commodities.main()

In [8]:
def _create_data():
    if DATA_SOURCE == 'sf1':
        from ml_investment.data_loaders.quandl_commodities import QuandlCommoditiesData
        from ml_investment.data_loaders.sf1 import SF1BaseData, SF1DailyData, \
                                                   SF1QuarterlyData
    elif DATA_SOURCE == 'mongo':
        from ml_investment.data_loaders.mongo import SF1BaseData, SF1DailyData, \
                                    SF1QuarterlyData, QuandlCommoditiesData        
    data = {}
    data['quarterly'] = SF1QuarterlyData()
    data['base'] = SF1BaseData()
    data['daily'] = SF1DailyData()
    data['commodities'] = QuandlCommoditiesData()
    
    return data

In [9]:
def _preprocess(x):
    x['rnd_invcap'] = x['rnd'] / x['invcap']
    x['capex_invcap'] = x['capex'] / x['invcap']
    x['ebit_invcap'] = x['ebit'] / x['invcap']
    x['ev_ebitda'] = x['ev'] / x['ebitda']
    x['ev_ebit'] = x['ev'] / x['ebit']
    x['debt_equity'] = x['debt'] / x['equity']
    x['grossmargin_ebitdamargin'] = x['grossmargin'] / x['ebitdamargin']
    x['debt_ebit'] = x['debt'] / x['ebitda']
       
    return x

In [10]:
def _create_feature():
    fc1 = QuarterlyFeatures(data_key='quarterly',
                            columns=QUARTER_COLUMNS + DEV_COLUMNS,
                            quarter_counts=QUARTER_COUNTS,
                            max_back_quarter=MAX_BACK_QUARTER,
                            min_back_quarter=MIN_BACK_QUARTER,
                            calc_stats_on_diffs=True,
                            data_preprocessing=_preprocess)
        
    fc2 = QuarterlyDiffFeatures(data_key='quarterly',
                                columns=QUARTER_COLUMNS + DEV_COLUMNS,
                                compare_quarter_idxs=COMPARE_QUARTER_IDXS,
                                max_back_quarter=MAX_BACK_QUARTER,
                                min_back_quarter=MIN_BACK_QUARTER,
                                data_preprocessing=_preprocess)

    fc3 = DailyAggQuarterFeatures(daily_data_key='commodities',
                                  quarterly_data_key='quarterly',
                                  columns=['price'],
                                  agg_day_counts=AGG_DAY_COUNTS,
                                  max_back_quarter=MAX_BACK_QUARTER,
                                  min_back_quarter=MIN_BACK_QUARTER,
                                  daily_index=COMMODITIES_CODES)
                      
    fc4 = RelativeGroupFeatures(feature_calculator=fc3,
                                group_data_key='base',
                                group_col='industry',
                                relation_foo=lambda x, y: x - y,
                                keep_group_feats=True)
    
    fc5 = RelativeGroupFeatures(feature_calculator=fc1,
                                group_data_key='base',
                                group_col='industry',
                                relation_foo=lambda x, y: x - y,
                                keep_group_feats=True)    
    
    feature = FeatureMerger(fc1, fc2, on=['ticker', 'date'])
    feature = FeatureMerger(feature, fc3, on=['ticker', 'date'])
    feature = FeatureMerger(feature, fc4, on=['ticker', 'date'])
    feature = FeatureMerger(feature, fc5, on=['ticker', 'date'])

    return feature

In [11]:
def _create_target():
    target = DailyDiffTarget(data_key='daily', col='marketcap')
    return target

In [ ]:
data = _create_data()
feature = _create_feature()
target = _create_target()
index = ['AAPL', 'TSLA']

X = feature.calculate(data, index)
y = target.calculate(self.data, X.index.to_frame(index=False))

In [33]:
from ml_investment.data_loaders.quandl_commodities import QuandlCommoditiesData
from ml_investment.data_loaders.sf1 import SF1BaseData, SF1DailyData, SF1QuarterlyData

quarterly = SF1QuarterlyData()
daily = SF1DailyData()
aapl_q = quarterly.load(['AAPl'])
aapl_d = daily.load(['AAPl'])

In [38]:
aapl_q  #[['date'] + QUARTER_COLUMNS]

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,assetsnc,assetturnover,bvps,capex,cashneq,cashnequsd,cor,consolinc,currentratio,de,debt,debtc,debtnc,debtusd,deferredrev,depamor,deposits,divyield,dps,ebit,ebitda,ebitdamargin,ebitdausd,ebitusd,ebt,eps,epsdil,epsusd,equity,equityavg,equityusd,ev,evebit,evebitda,fcf,fcfps,fxusd,gp,grossmargin,intangibles,intexp,invcap,invcapavg,inventory,investments,investmentsc,investmentsnc,liabilities,liabilitiesc,liabilitiesnc,marketcap,ncf,ncfbus,ncfcommon,ncfdebt,ncfdiv,ncff,ncfi,ncfinv,ncfo,ncfx,netinc,netinccmn,netinccmnusd,netincdis,netincnci,netmargin,opex,opinc,payables,payoutratio,pb,pe,pe1,ppnenet,prefdivis,price,ps,ps1,receivables,retearn,revenue,revenueusd,rnd,roa,roe,roic,ros,sbcomp,sgna,sharefactor,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital,date
0,AAPL,ARQ,2021-06-30,2021-07-28,2021-06-26,2021-07-28,58000000,329840000000,NaN,114423000000,215417000000,NaN,3.865,-2093000000,34050000000,34050000000,46179000000,21744000000,1.062,4.131,121791000000,16039000000,105752000000,121791000000,7681000000,2832000000,0,0.006,0.220,24369000000,27201000000,0.334,27201000000,24369000000,24369000000,1.310,1.300,1.310,64280000000,NaN,64280000000,2513873463820,25,22.474,19001000000,1.143,1.0,35255000000,0.433,0,0,309827000000,NaN,5178000000,159594000000,27646000000,131948000000,265560000000,107754000000,157806000000,2426132463820,-4730000000,-4000000,-22900000000,0,-3767000000,-29396000000,3572000000,5747000000,21094000000,0,21744000000,21744000000,21744000000,0,0,0.267,11129000000,24126000000,40409000000,0.168,37.743,27.950,28.400,38615000000,0,146.770,6.989,7.031,33908000000,9233000000,81434000000,81434000000,5717000000,NaN,NaN,NaN,NaN,1960000000,5412000000,1.0,16530166000,16629371000,16781735000,4.897,329840000000,0,2625000000,0,19.835,6669000000,2021-07-28
1,AAPL,ARQ,2021-03-31,2021-04-29,2021-03-27,2021-07-28,-286000000,337158000000,NaN,121465000000,215693000000,NaN,4.129,-2269000000,38466000000,38466000000,51505000000,23630000000,1.142,3.874,121645000000,13003000000,108642000000,121645000000,7595000000,2797000000,0,0.006,0.205,28011000000,30808000000,0.344,30808000000,28011000000,28011000000,1.410,1.400,1.410,69178000000,NaN,69178000000,2310643985880,26,22.981,21712000000,1.296,1.0,38079000000,0.425,0,0,313952000000,NaN,5219000000,165907000000,31368000000,134539000000,267980000000,106385000000,161595000000,2227464985880,2287000000,0,-17987000000,10423000000,-3447000000,-11326000000,-10368000000,-7895000000,23981000000,0,23630000000,23630000000,23630000000,0,0,0.264,10576000000,27503000000,40127000000,0.145,32.199,29.189,29.596,37815000000,0,133.480,6.845,6.872,33036000000,15261000000,89584000000,89584000000,5262000000,NaN,NaN,NaN,NaN,1981000000,5314000000,1.0,16687631000,16753476000,16929157000,5.347,337158000000,0,4381000000,0,20.125,15080000000,2021-04-29
2,AAPL,ARQ,2020-12-31,2021-01-28,2020-12-26,2021-07-28,179000000,354054000000,NaN,154106000000,199948000000,NaN,3.910,-3500000000,36010000000,36010000000,67111000000,28755000000,1.163,4.346,112043000000,12762000000,99281000000,112043000000,7395000000,2666000000,0,0.006,0.205,33579000000,36245000000,0.325,36245000000,33579000000,33579000000,1.700,1.680,1.700,66224000000,NaN,66224000000,2377513080640,32,27.756,35263000000,2.082,1.0,44328000000,0.398,0,0,297580000000,NaN,4973000000,159561000000,40816000000,118745000000,287830000000,132507000000,155323000000,2301480080640,-2070000000,-9000000,-24775000000,-978000000,-3613000000,-32249000000,-8584000000,-5279000000,38763000000,0,28755000000,28755000000,28755000000,0,0,0.258,10794000000,33534000000,63846000000,0.121,34.753,36.000,36.606,37933000000,0,137.090,7.825,7.893,58620000000,14301000000,111439000000,111439000000,5163000000,NaN,NaN,NaN,NaN,2020000000,5631000000,1.0,16788096000,16935119000,17113688000,6.580,354054000000,0,4824000000,0,20.906,21599000000,2021-01-28
3,AAPL,ARQ,2020-09-30,2020-1

In [67]:
aapl_d[50:115]
(2426132463820 - 2227464985880) / 2227464985880

0.08918994426370876

In [12]:
def _create_model():
#     base_models = [lgbm.sklearn.LGBMRegressor(),
#                    ctb.CatBoostRegressor(verbose=False)]
                   
#     ensemble = EnsembleModel(base_models=base_models, 
#                              bagging_fraction=BAGGING_FRACTION,
#                              model_cnt=MODEL_CNT)

#     model = GroupedOOFModel(ensemble,
#                             group_column='ticker',
#                             fold_cnt=FOLD_CNT)
    
    base_model = ctb.CatBoostRegressor(verbose=False)
    
    model = GroupedOOFModel(base_model,
                            group_column='ticker',
                            fold_cnt=FOLD_CNT)
    
    return model

In [13]:
def FairMarketcapDiffSF1V2(max_back_quarter: int=None,
                           min_back_quarter: int=None,
                           data_source: Optional[str]=None,
                           pretrained: bool=True) -> Pipeline:
    if data_source is not None:
        global DATA_SOURCE 
        DATA_SOURCE = data_source
        
    if max_back_quarter is not None:
        global MAX_BACK_QUARTER 
        MAX_BACK_QUARTER = max_back_quarter

    if min_back_quarter is not None:
        global MIN_BACK_QUARTER 
        MIN_BACK_QUARTER = min_back_quarter

    if DATA_SOURCE == 'sf1':
        _check_download_data()
        
    data = _create_data()
    feature = _create_feature()
    target = _create_target()
    model = _create_model()

    pipeline = Pipeline(feature=feature, 
                        target=target, 
                        model=model,
                        data=data,
                        out_name=OUT_NAME)
            
    core_path = '{}/{}.pickle'.format(config['models_path'], OUT_NAME)

    if pretrained:       
        pipeline.load_core(core_path)

    return pipeline

In [15]:
pipeline = FairMarketcapDiffSF1V2(pretrained=False, data_source=DATA_SOURCE)    
base_df = pipeline.data['base'].load()
tickers = base_df[(base_df['currency'] == CURRENCY) &\
                  (base_df['scalemarketcap'].apply(lambda x: x in SCALE_MARKETCAP))
                 ]['ticker'].values

filter_foo = bound_filter_foo_gen(min_bound=MIN_TARGET_BOUND,
                                  max_bound=MAX_TARGET_BOUND)

result = pipeline.fit(tickers,
                      metric=median_abs_diff,
                      target_filter_foo=filter_foo)
print(result)
path = '{}/{}'.format(config['models_path'], OUT_NAME)
pipeline.export_core(path)    

0it [00:00, ?it/s]Process SpawnPoolWorker-2:
Process SpawnPoolWorker-4:
Process SpawnPoolWorker-3:
Process SpawnPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task

Process SpawnPoolWorker-13:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute '_preprocess' on <module '__main__' (built-in)>
Process SpawnPoolWorker-14:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.

Process SpawnPoolWorker-26:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute '_preprocess' on <module '__main__' (built-in)>
Process SpawnPoolWorker-25:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.

Process SpawnPoolWorker-40:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute '_preprocess' on <module '__main__' (built-in)>
Process SpawnPoolWorker-37:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.

Process SpawnPoolWorker-49:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute '_preprocess' on <module '__main__' (built-in)>
Process SpawnPoolWorker-50:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.

...

Process SpawnPoolWorker-121:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute '_preprocess' on <module '__main__' (built-in)>
Process SpawnPoolWorker-122:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/

Process SpawnPoolWorker-133:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute '_preprocess' on <module '__main__' (built-in)>
Process SpawnPoolWorker-134:
Process SpawnPoolWorker-135:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    s

Process SpawnPoolWorker-146:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute '_preprocess' on <module '__main__' (built-in)>
Process SpawnPoolWorker-145:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/

  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute '_preprocess' on <module '__main__' (built-in)>
Process SpawnPoolWorker-160:
0it [00:54, ?it/s]Process SpawnPoolWorker-159:



KeyboardInterrupt: 